In [14]:
pip install PyPDF2==2.12.1

   ---------------------------------------- 0.0/222.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/222.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/222.8 kB ? eta -:--:--
   ------- ------------------------------- 41.0/222.8 kB 495.5 kB/s eta 0:00:01
   ---------------------------------- ----- 194.6/222.8 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 222.8/222.8 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: PyPDF2
    Found existing installation: PyPDF2 3.0.1
    Uninstalling PyPDF2-3.0.1:
      Successfully uninstalled PyPDF2-3.0.1
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install easyocr

   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.9 MB 5.6 MB/s eta 0:00:01
   ------- -------------------------------- 0.5/2.9 MB 6.7 MB/s eta 0:00:01
   ---------------- ----------------------- 1.2/2.9 MB 9.5 MB/s eta 0:00:01
   --------------------------------- ------ 2.4/2.9 MB 14.1 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/313.0 kB ? eta -:--:--
   --------------------------------------- 313.0/313.0 kB 18.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   - -------------------------------------- 1.4/38.5 MB 45.6 MB/s eta 0:00:01
   --- ------------------------------------ 3.1/38.5 MB 39.5 MB/s eta 0:00:01
   ---- ----------------------------------- 4.7/38.5 MB 37.8 MB/s eta 0:00:01
   ------ --------------------------------- 6.4/38.5 MB 37.3 MB/s eta 0:00:01
   -------- ----

In [11]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install PIP

Note: you may need to restart the kernel to use updated packages.


In [8]:
import PyPDF2
import os
from PIL import Image
import io

final={}

def extract_images_from_pdf(pdf_path, output_path):
    pdf_document = PyPDF2.PdfFileReader(pdf_path)
    for current_page_number in range(pdf_document.getNumPages()):
        current_page = pdf_document.getPage(current_page_number)
        if "/XObject" in current_page["/Resources"]:
            x_object = current_page["/Resources"]["/XObject"].getObject()
            for obj in x_object:
                if x_object[obj]["/Subtype"] == "/Image":
                    image = x_object[obj]
                    image_bytes = image._data
                    image = Image.open(io.BytesIO(image_bytes))
                    image.save(os.path.join(output_path, f"{current_page_number}.png"))

def ocr_on_images(images_folder_path):
    reader = easyocr.Reader(["en"])
    for image_name in os.listdir(images_folder_path):
        image_path = os.path.join(images_folder_path, image_name)
        image = Image.open(image_path)

        #crop the image to get the required coordinates x1=668, y1=1112, x2=2740, y2=1390
        image = image.crop((668, 1112, 2740, 1390))
        image = np.array(image)
        result = reader.readtext(image)
        
        # for detection in result:
        #     if detection[0][1] >= 668 and detection[0][3] <= 1112:
        #         print(f"Text between coordinates 668, 1112, 2740, 1390 is: {detection[1]}")
        if result==[]:
            print(image_name[0], "No text found")
            
        else:
            print(image_name[0], result[0][1])

        final[image_name[0]]="No text found" if result==[] else result[0][1]


    print(final)
        # sort the dictionary according to keys
    key_list=sorted(final.keys())
    sorted_dict = {}
    for key in key_list:
        sorted_dict[key] = final[key]

    print(sorted_dict)

    flg=0


    for key in sorted_dict:
        if sorted_dict[key]=="COMPLAINT":
            #store {key}.png in complaint folder
            image = Image.open(f"./output/{key}.png")
            image.save(f"./complaint/{key}.png")
            flg=1

        elif sorted_dict[key]=="SUMMONS":
            #store {key}.png in summons folder
            image = Image.open(f"./output/{key}.png")
            image.save(f"./summons/{key}.png")
            flg=2

        elif sorted_dict[key]=="Appearance By Attorney In Civil Case":
            #store {key}.png in attorney folder
            image = Image.open(f"./output/{key}.png")
            image.save(f"./appearance/{key}.png")
            flg=3

        else:
            if flg==1:
                image = Image.open(f"./output/{key}.png")
                image.save(f"./complaint/{key}.png")

            elif flg==2:
                image = Image.open(f"./output/{key}.png")
                image.save(f"./summons/{key}.png")

            elif flg==3:
                image = Image.open(f"./output/{key}.png")
                image.save(f"./appearance/{key}.png")

def images_to_pdf(folder_path, output_pdf):
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
    images = [Image.open(image) for image in image_paths]
    images[0].save(output_pdf, save_all=True, append_images=images[1:])

if __name__ == "__main__":
    pdf_path = "123456.pdf"
    output_path = "./output"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    extract_images_from_pdf(pdf_path, output_path)
    print(f"Images extracted from {pdf_path} and saved in {output_path} folder")
    
    images_folder_path = "./output"
    ocr_on_images(images_folder_path)

    # Naming output PDF files
    input_pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]  # Extracting input PDF file name without extension
    folders = ["complaint", "summons", "appearance"]
    for folder in folders:
        output_pdf = f"{input_pdf_name}_{folder}.pdf"
        images_to_pdf(folder, output_pdf)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Images extracted from 123456.pdf and saved in ./output folder
0 COMPLAINT
1 No text found
2 No text found
3 SUMMONS
4 Appearance By Attorney In Civil Case
{'0': 'COMPLAINT', '1': 'No text found', '2': 'No text found', '3': 'SUMMONS', '4': 'Appearance By Attorney In Civil Case'}
{'0': 'COMPLAINT', '1': 'No text found', '2': 'No text found', '3': 'SUMMONS', '4': 'Appearance By Attorney In Civil Case'}
